In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Usando dispositivo:", device)

Usando dispositivo: cuda


In [24]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(), 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])
transform_test = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [26]:
def apply_mask(net, individuo):
    for (param_net, param_ind) in zip(net.parameters(), individuo.parameters()):
        flat = param_ind.data.view(-1)
        k = int(0.1 * flat.numel())
        topk_vals, _ = torch.topk(flat.abs(), k)
        threshold = topk_vals[-1]
        mask = (param_ind.data.abs() >= threshold).float()
        param_net.data *= mask

def train_net(net, train_loader, optimizer, criterion, device):
    net.train()
    for epoch in range(10):
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

def evaluate_net(net, test_loader, criterion, device):
    net.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss += criterion(outputs, targets).item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return loss / len(test_loader), correct / total

def improvements(loss_list):
    last_10 = loss_list[-10:]
    return sum(1 for i in range(9) if last_10[i+1] < last_10[i])

In [ ]:
individuo = AlexNet().to(device)
varianzas = AlexNet().to(device)
net = AlexNet().to(device)
for param in varianzas.parameters():
    param.data.fill_(0.5)

losses = []
accuracies = []
epoch = 1

while True:
    apply_mask(net, individuo)
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    train_net(net, train_loader, optimizer, criterion, device)
    loss, acc = evaluate_net(net, test_loader, criterion, device)
    losses.append(loss)
    accuracies.append(acc)
    print(f"Epoch {epoch} - Loss: {loss:.4f}, Acc: {acc:.4f}")

    if epoch == 1 or loss < losses[-2]:
        individuo.load_state_dict(individuo.state_dict())

    if acc >= 0.5:
        print("Accuracy goal reached.")
        break

    if epoch >= 10:
        imp = improvements(losses)
        factor = 1.0 if imp == 2 else (0.82 if imp < 2 else 1/0.82)
        for param in varianzas.parameters():
            param.data *= factor

    nuevo_ind = AlexNet().to(device)
    with torch.no_grad():
        for p_new, p_old, p_var in zip(nuevo_ind.parameters(), individuo.parameters(), varianzas.parameters()):
            noise = torch.normal(0, torch.abs(p_var))
            p_new.copy_(p_old + noise)
    individuo = nuevo_ind
    epoch += 1


Epoch 1 - Loss: 0.7341, Acc: 0.7412
Accuracy goal reached.


: 